In [4]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install gmaps

     |████████████████████████████████| 1.1 MB 4.2 MB/s eta 0:00:01


  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076111 sha256=22e0a99433978901150f9e470a215cb3aa28cfb7235575bc070d597bc34a2978
  Stored in directory: /Users/akarim/Library/Caches/pip/wheels/66/ab/29/3032938612273665f0fa4d2fab4e3ecfe86480eaaa40aaee91
Successfully built gmaps


In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Great Falls,US,2022-08-29 16:43:44,47.5002,-111.3008,72.41,35,0,11.50
1,1,Codrington,AU,2022-08-29 16:43:45,-38.2667,141.9667,47.75,98,100,10.58
2,2,Sept-Iles,CA,2022-08-29 16:43:45,50.2001,-66.3821,73.40,72,75,10.36
3,3,Bluff,NZ,2022-08-29 16:43:45,-46.6000,168.3333,44.80,85,56,4.76
4,4,Port Alfred,ZA,2022-08-29 16:43:46,-33.5906,26.8910,53.19,86,100,4.00


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 50


In [9]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Codrington,AU,2022-08-29 16:43:45,-38.2667,141.9667,47.75,98,100,10.58
3,3,Bluff,NZ,2022-08-29 16:43:45,-46.6000,168.3333,44.80,85,56,4.76
11,11,Geraldton,AU,2022-08-29 16:43:47,-28.7667,114.6000,45.01,87,0,8.05
16,16,Kodiak,US,2022-08-29 16:43:49,57.7900,-152.4072,49.51,87,100,5.75
20,20,Aklavik,CA,2022-08-29 16:42:20,68.2191,-135.0107,44.98,80,99,6.73


In [10]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [11]:
preferred_cities_df.count()

City_ID       84
City          84
Country       84
Date          84
Lat           84
Lng           84
Max Temp      84
Humidity      84
Cloudiness    84
Wind Speed    84
dtype: int64

In [12]:
preferred_cities_df = preferred_cities_df.dropna()

In [13]:
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Codrington,AU,2022-08-29 16:43:45,-38.2667,141.9667,47.75,98,100,10.58
3,3,Bluff,NZ,2022-08-29 16:43:45,-46.6000,168.3333,44.80,85,56,4.76
11,11,Geraldton,AU,2022-08-29 16:43:47,-28.7667,114.6000,45.01,87,0,8.05
16,16,Kodiak,US,2022-08-29 16:43:49,57.7900,-152.4072,49.51,87,100,5.75
20,20,Aklavik,CA,2022-08-29 16:42:20,68.2191,-135.0107,44.98,80,99,6.73


In [14]:
hotel_df = preferred_cities_df[["City","Country","Max Temp","Lat","Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Codrington,AU,47.75,-38.2667,141.9667,
3,Bluff,NZ,44.80,-46.6000,168.3333,
11,Geraldton,AU,45.01,-28.7667,114.6000,
16,Kodiak,US,49.51,57.7900,-152.4072,
20,Aklavik,CA,44.98,68.2191,-135.0107,
28,Bredasdorp,ZA,48.81,-34.5322,20.0403,
34,Saskylakh,RU,40.53,71.9167,114.0833,
42,Upernavik,GL,38.08,72.7868,-56.1549,
45,Ushuaia,AR,44.26,-54.8000,-68.3000,
47,Mataura,NZ,44.47,-46.1927,168.8643,


In [15]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [16]:
#Iterate through the df

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    #Add the latitude and longitude to location key for param dict
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found.... skipping.")

Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.


In [17]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Codrington,AU,47.75,-38.2667,141.9667,Codrington Gardens Bed And Breakfast
3,Bluff,NZ,44.80,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
11,Geraldton,AU,45.01,-28.7667,114.6000,Broadwater Mariner Resort
16,Kodiak,US,49.51,57.7900,-152.4072,Best Western Kodiak Inn
20,Aklavik,CA,44.98,68.2191,-135.0107,Aklavik Inn
28,Bredasdorp,ZA,48.81,-34.5322,20.0403,Bredasdorp Country Manor
34,Saskylakh,RU,40.53,71.9167,114.0833,
42,Upernavik,GL,38.08,72.7868,-56.1549,Café de Upernavik
45,Ushuaia,AR,44.26,-54.8000,-68.3000,Albatross Hotel
47,Mataura,NZ,44.47,-46.1927,168.8643,Ellie's Villa


In [18]:
locations = hotel_df[["Lat" , "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating = False, max_intensity = 300, point_radius = 4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [20]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))